# Just Experimenting: Skip Section to Next

This section is just exploration and getting my bearings

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from monai.transforms import LoadImaged, MapTransform, Compose, ToNumpyd
from monai.data import Dataset


In [26]:
import glob
Validation_Cases = list(glob.glob("./MICCAI_BraTS2020_TrainingData/BraTS20*"))

In [28]:
experiment_cases = []

for case in Validation_Cases:
    try:
        t1 = list(glob.glob(f"{case}/*t1.nii.gz"))
        t1ce = list(glob.glob(f"{case}/*t1ce.nii.gz"))
        t2 = list(glob.glob(f"{case}/*t2.nii.gz"))
        flair = list(glob.glob(f"{case}/*flair.nii.gz"))
        seg = glob.glob(f"{case}/*seg.nii.gz")[0]

        img = t1 + t1ce + t2 + flair

        d = {"image": img, "seg": seg}
        experiment_cases.append(d)
    except Exception as e:
        print(e)
        print(case)


In [30]:
class ConvertLabelsIntoOneHotd(MapTransform):
    """Takes input tensor of segmentation which contains
    values in set (0,1,2,4) where\n
    0 -> Background/Normal\n
    1 -> Non- Enhancing Tumor Core\n
    2 -> Edema\n
    4 -> Enhancing tumor core\n

    and returns a one hot encoded 3 channel tensor where
    1st Channel -> Whole tumor (1,2 and 4)
    2nd Channel -> Tumor Core (1 and 4)
    3rd Channel -> Enhancing Tumor core (4)
    """
    def __call__(self, data):
        data_dict = dict(data)
        for key in self.keys:
            one_hot_encode_array = [
                torch.logical_or(
                    torch.logical_or(data_dict[key] == 1, data_dict[key] == 2),
                    data_dict[key] == 4,
                ), # Whole Tumor
                torch.logical_or(data_dict[key] == 1, data_dict[key] == 4), # Tumor Core
                data_dict[key] == 4, # Enhancing Core
            ]
            data_dict[key] = torch.stack(one_hot_encode_array, axis=0).astype(torch.int32)
        return data_dict

In [31]:
transform_for_analysis = Compose(
    [
        LoadImaged(keys=["image", "seg"]),
        ConvertLabelsIntoOneHotd(keys="seg"),
        ToNumpyd(keys=["image", "seg"])
    ]
)


In [32]:
dataset = Dataset(experiment_cases,transform_for_analysis)

## Textural Analysis

1. Take GLCM from Tumor Boundary

In [1]:
# functions to find boundary from segmentation
from scipy import ndimage

In [34]:

case = 25
wt = dataset[case]['seg'][0]
tc = dataset[case]['seg'][1]
et = dataset[case]['seg'][2]

t1 = dataset[case]['image'][0]
t1ce = dataset[case]['image'][1]
t2 = dataset[case]['image'][2]
flair = dataset[case]['image'][3]


In [35]:
blob_wt = ndimage.find_objects(wt)
print(f"{len(blob_wt)} WT blob found")
blob_tc = ndimage.find_objects(tc)
print(f"{len(blob_tc)} TC blob found")
blob_et = ndimage.find_objects(et)
print(f"{len(blob_et)} ET blob found")

1 WT blob found
1 TC blob found
1 ET blob found


In [37]:
# Finding Center of Mass and Volume

print("For Whole Tumor")
for i,blob in enumerate(blob_wt):
    print(f"Blob {i}: coordinate {ndimage.center_of_mass(wt[blob])} and {np.count_nonzero(wt[blob]) / 10**3} cm^3")


print("\nFor Tumor Core")
for i,blob in enumerate(blob_tc):
    print(f"Blob {i}: coordinate {ndimage.center_of_mass(tc[blob])} and {np.count_nonzero(tc[blob]) / 10**3} cm^3")


print("\nFor Enhancing Tumor")
for i,blob in enumerate(blob_et):
    print(f"Blob {i}: coordinate {ndimage.center_of_mass(et[blob])} and {np.count_nonzero(et[blob]) / 10**3} cm^3")

For Whole Tumor
Blob 0: coordinate (30.557043041114397, 53.65276439778981, 42.876817371469066) and 180.618 cm^3

For Tumor Core
Blob 0: coordinate (21.879299974221638, 32.3802308595652, 25.608541231060062) and 34.913 cm^3

For Enhancing Tumor
Blob 0: coordinate (23.28733738807231, 32.01900658895084, 27.14660415610745) and 23.676 cm^3


In [40]:
# GLCM

# Based on https://github.com/Prof-Iz/GLCM-from-3D-NumPy-Input

def glcm_3d(input: np.ndarray, delta: tuple[int] = (1, 1, 1), d: int = 1):
    """_summary_

    Args:
        input (np.ndarray): input array. 3D. dtype int
        delta (tuple[int], optional): Direction vector from pixel. Defaults to (1, 1, 1).
        d (int, optional): Distance to check for neighbouring channel. Defaults to 1.

    Raises:
        Exception: if input is not of type dint or is not 3D

    Returns:
        _type_: GLCM Matrix
    """

    if 'int' not in input.dtype.__str__():
        raise Exception("Input should be of dtype Int")

    if len(input.shape) != 3:
        raise Exception("Input should be 3 dimensional")

    offset = (delta[0] * d, delta[1] * d, delta[2] * d)  # offset from each pixel

    x_max, y_max, z_max = input.shape  # boundary conditions during enumeration

    levels = input.max() + 1 # 0:1:n assume contn range of pixel values

    print(f"levels : {levels}")
    results = np.zeros((levels, levels))  # initialise results error


    for i, v in np.ndenumerate(input):
        x_offset = i[0] + offset[0]
        y_offset = i[1] + offset[1]
        z_offset = i[2] + offset[2]

        if (x_offset >= x_max) or (y_offset >= y_max) or (z_offset >= z_max):
            # if offset out of boundary skip
            continue

        value_at_offset = input[x_offset, y_offset, z_offset]

        results[v, value_at_offset] += 1

    return results / levels**2

In [45]:
# find GLCM for a blob in WT

wt_tumor_boundary_t1    = t1[blob_wt[0]]
wt_tumor_boundary_t1ce  = t1ce[blob_wt[0]]
wt_tumor_boundary_t2    = t2[blob_wt[0]]
wt_tumor_boundary_flair = flair[blob_wt[0]]

wt_tumor_boundary_t1.shape

(66, 110, 79)

In [91]:
print(graycoprops(glcm))

{'energy': 0.010458012361037416, 'homogeneity': 0.14261134493343397, 'contrast': 7161.607885927293, 'entropy': 10.093394917491178, 'correlation': 0.9076729367777118}


# Defining Functions for each feature

In [2]:
from scipy import ndimage
import numpy as np

In [3]:
# GLCM

# Based on https://github.com/Prof-Iz/GLCM-from-3D-NumPy-Input


def glcm_3d(input: np.ndarray, delta: tuple[int] = (1, 1, 1), d: int = 1):
    """_summary_

    Args:
        input (np.ndarray): input array. 3D. dtype int
        delta (tuple[int], optional): Direction vector from pixel. Defaults to (1, 1, 1).
        d (int, optional): Distance to check for neighbouring channel. Defaults to 1.

    Raises:
        Exception: if input is not of type dint or is not 3D

    Returns:
        _type_: GLCM Matrix
    """

    if "int" not in input.dtype.__str__():
        raise Exception("Input should be of dtype Int")

    if len(input.shape) != 3:
        raise Exception("Input should be 3 dimensional")

    offset = (delta[0] * d, delta[1] * d, delta[2] * d)  # offset from each pixel

    x_max, y_max, z_max = input.shape  # boundary conditions during enumeration

    levels = input.max() + 1  # 0:1:n assume contn range of pixel values

    results = np.zeros((levels, levels))  # initialise results error

    for i, v in np.ndenumerate(input):
        x_offset = i[0] + offset[0]
        y_offset = i[1] + offset[1]
        z_offset = i[2] + offset[2]

        if (x_offset >= x_max) or (y_offset >= y_max) or (z_offset >= z_max):
            # if offset out of boundary skip
            continue

        value_at_offset = input[x_offset, y_offset, z_offset]

        results[v, value_at_offset] += 1

    return results / levels**2


def normalise(P):
    return P / np.sum(P)


def graycoprops(P, debug=False):

    P = normalise(P)

    energy = np.sum(P**2)

    idx = np.where(P > 0)

    # begin i , j indexing from 1
    i = idx[0] + 1
    j = idx[1] + 1

    homogeneity = np.sum(P[idx] / (1 + (i - j) ** 2))

    contrast = np.sum(P[idx] * (i - j) ** 2)

    entropy = np.sum(-P[idx] * np.log(P[idx]))

    # for correlation

    mu = np.sum(i * P[idx])

    sigma_square = np.sum(P[idx] * (i - mu) ** 2)

    correlation = np.sum((P[idx] * (i - mu) * (j - mu)) / sigma_square)

    if debug:
        print(
            f"""
        i: {i}\n
        j: {j}\n
        mu: {mu}\n
        sigma_square: {sigma_square}\n
        """
        )

    return {
        "energy": energy,
        "homogeneity": homogeneity,
        "contrast": contrast,
        "entropy": entropy,
        "correlation": correlation,
    }


In [4]:
def get_features(input):
    seg = np.squeeze(np.uint32(input["seg"]))
    scan = np.squeeze(np.uint32(input["image"]))

    wt = seg[0]
    tc = seg[1]
    et = seg[2]

    t1 = scan[0]
    t1ce = scan[1]
    t2 = scan[2]
    flair = scan[3]

    feature = {}
    blob_wt = ndimage.find_objects(wt)
    blob_tc = ndimage.find_objects(tc)
    blob_et = ndimage.find_objects(et)

    
    feature['blobs'] = {
        "blob_wt" : len(blob_wt),
        "blob_tc" : len(blob_tc),
        "blob_et" : len(blob_et),

    }


    # assume 0th index of blob is largest one






    glcm_t1 = glcm_3d(t1[blob_wt[0]])
    glcm_t1ce = glcm_3d(t1ce[blob_wt[0]])
    glcm_t2 = glcm_3d(t2[blob_wt[0]])
    glcm_flair = glcm_3d(flair[blob_wt[0]])

    # Textural analysis on Largest WT blob
    feature['textural'] = {
        't1' : graycoprops(glcm_t1),
        't1ce' : graycoprops(glcm_t1ce),
        't2' : graycoprops(glcm_t2),
        'flair' : graycoprops(glcm_flair)
    }


    
    wt_non_txt = {
            "center": ndimage.center_of_mass(wt[blob_wt[0]]),  # center of mass
            "volume": np.count_nonzero(wt[blob_wt[0]]) / 10**3,  # volume in cm^3
        }
    tc_non_txt = {
            "center": ndimage.center_of_mass(tc[blob_tc[0]]),  # center of mass
            "volume": np.count_nonzero(tc[blob_tc[0]]) / 10**3,  # volume in cm^3
        }
    et_non_txt = {
            "center": ndimage.center_of_mass(et[blob_et[0]]),  # center of mass
            "volume": np.count_nonzero(et[blob_et[0]]) / 10**3,  # volume in cm^3
        }

    feature['non-textural'] = {
        "wt" : wt_non_txt,
        "tc" : tc_non_txt,
        "et" : et_non_txt
    }

    return feature

    

# Calculating for Ground Truth Vs. Prediction

In [5]:
import torch
from monai.inferers import sliding_window_inference
from monai.transforms import (
    Activations,
    AsDiscrete,
    LoadImaged,
    Compose,
    MapTransform,
    EnsureChannelFirstd,
    ToMetaTensord,
    Orientationd,
    NormalizeIntensityd,
)
from monai.networks.nets import UNet
from monai.metrics import DiceMetric, HausdorffDistanceMetric
import glob


In [6]:
device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

model = UNet(
    spatial_dims=3,
    in_channels=4,
    out_channels=3,
    strides=(2, 2, 2),
    channels=[16,32,64,128],
    num_res_units=2,
).to(device)

chk = torch.load("./experiments/Patch_Size_Experiment_Results/best_metric_dyn_unet_dicece_2res_chan128div3_const_sliding.pth")


model.load_state_dict(chk["model"])

<All keys matched successfully>

In [6]:
import glob
Validation_Cases = list(glob.glob("./MICCAI_BraTS2020_TrainingData/*"))

experiment_cases = []

for case in Validation_Cases:
    t1 = list(glob.glob(f"{case}/*t1.nii.gz"))
    t1ce = list(glob.glob(f"{case}/*t1ce.nii.gz"))
    t2 = list(glob.glob(f"{case}/*t2.nii.gz"))
    flair = list(glob.glob(f"{case}/*flair.nii.gz"))
    seg = glob.glob(f"{case}/*seg.nii.gz")[0]

    img = t1 + t1ce + t2 + flair

    d = {"image": img, "seg": seg}
    experiment_cases.append(d)

    break

In [7]:
class ConvertLabelsIntoOneHotd(MapTransform):
    """Takes input tensor of segmentation which contains
    values in set (0,1,2,4) where\n
    0 -> Background/Normal\n
    1 -> Non- Enhancing Tumor Core\n
    2 -> Edema\n
    4 -> Enhancing tumor core\n

    and returns a one hot encoded 3 channel tensor where
    1st Channel -> Whole tumor (1,2 and 4)
    2nd Channel -> Tumor Core (1 and 4)
    3rd Channel -> Enhancing Tumor core (4)
    """

    def __call__(self, data):
        data_dict = dict(data)
        for key in self.keys:
            one_hot_encode_array = [
                torch.logical_or(
                    torch.logical_or(data_dict[key] == 1, data_dict[key] == 2),
                    data_dict[key] == 4,
                ),  # Whole Tumor
                torch.logical_or(
                    data_dict[key] == 1, data_dict[key] == 4
                ),  # Tumor Core
                data_dict[key] == 4,  # Enhancing Core
            ]
            data_dict[key] = torch.stack(one_hot_encode_array, axis=0).astype(
                torch.float32
            )
        return data_dict


transform_no_norm = Compose(
    [
        LoadImaged(keys=["image", "seg"]),
        EnsureChannelFirstd(keys=["image"]),
        ConvertLabelsIntoOneHotd(keys="seg"),
        ToMetaTensord(keys=["image", "seg"]),
        Orientationd(keys=["image", "seg"], axcodes="RAS"),
        #! Training Dataset Already set to Pixel Dimension of 1mm^3
    ]
)

normalise_mri = NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True)


In [8]:
def inference(input):
        """Do Sliding Window Inference on input tensor
        To avoid OOM Error, Input Model done on CPU.
        Patch taken from input and its inference done on GPU
        to speed up inference time.

        Args:
            input: Full input to pass in the model. For the case
            of this project size => (3,240,240,155)
        """

        def _compute(input):
            return sliding_window_inference(
                inputs=input.to("cpu"),
                roi_size=(128,128,128),
                sw_batch_size=1,
                predictor=model,
                overlap=0.5,
                padding_mode="constant",
                sw_device="cuda:0",
                device="cpu",
            )

        with torch.cuda.amp.autocast():
            return _compute(input)

dice_metric = DiceMetric(include_background=True, reduction="mean")
dice_metric_batch = DiceMetric(include_background=True, reduction="mean_batch")

hausdorff_metric = HausdorffDistanceMetric(
    include_background=True, distance_metric='euclidean',
    reduction="mean"
)

hausdorff_metric_batch = HausdorffDistanceMetric(
    include_background=True, distance_metric='euclidean',
    reduction="mean_batch"
)
post_processing_validation = Compose(
    [Activations(sigmoid=True), AsDiscrete(threshold=0.5)]
)

In [10]:
def evaluate_model(model, case, TRAINING_DATASET_PATH = r"./MICCAI_BraTS2020_TrainingData"):

    t1 = list(glob.glob(f"{TRAINING_DATASET_PATH}/*{case:03}/*t1.nii.gz"))
    t1ce = list(glob.glob(f"{TRAINING_DATASET_PATH}/*{case:03}/*t1ce.nii.gz"))
    t2 = list(glob.glob(f"{TRAINING_DATASET_PATH}/*{case:03}/*t2.nii.gz"))
    flair = list(glob.glob(f"{TRAINING_DATASET_PATH}/*{case:03}/*flair.nii.gz"))
    seg = glob.glob(f"{TRAINING_DATASET_PATH}/*{case:03}/*seg.nii.gz")

    img = t1 + t1ce + t2 + flair

    d = {"image": img, "seg": seg}

    val_data_input = transform_no_norm(d)

    val_data = {}
    val_data['image'] = val_data_input['image'].detach().clone()
    val_data['seg'] = val_data_input['seg'].detach().clone()
    

    val_data = normalise_mri(val_data)


    model.eval()
    with torch.no_grad():

        val_inputs, val_labels = (
            val_data["image"],
            val_data["seg"].to(device),
        )

        val_outputs = inference(torch.unsqueeze(val_inputs,0))
        
        val_labels = torch.unsqueeze(val_labels,0)

        val_outputs_post = post_processing_validation(val_outputs.to(device))

        dice_metric(y_pred=val_outputs_post, y=val_labels)
        dice_metric_batch(y_pred=val_outputs_post, y=val_labels)

        hausdorff_metric(y_pred=val_outputs_post, y=val_labels)
        hausdorff_metric_batch(y_pred=val_outputs_post, y=val_labels)



        dice_avg = dice_metric.aggregate().item()
        dice_batch = dice_metric_batch.aggregate()
        dice_wt = dice_batch[0].item()
        dice_tc = dice_batch[1].item()
        dice_et = dice_batch[2].item()

        dice_metric.reset()

        dice_metric_batch.reset()


        
        hausdorff_avg = hausdorff_metric.aggregate().item()
        hausdorff_batch = hausdorff_metric_batch.aggregate()
        hausdorff_wt = hausdorff_batch[0].item()
        hausdorff_tc = hausdorff_batch[1].item()
        hausdorff_et = hausdorff_batch[2].item()


        hausdorff_metric.reset()
        hausdorff_metric_batch.reset()




        return {
            "input" : val_data_input,
            "pred": val_outputs_post,
            "metrics": {
                "case" : [case],
                "dice_avg": [dice_avg],
                "dice_wt" : [dice_wt],
                "dice_tc" :[dice_tc],
                "dice_et" : [dice_et],
                "hdf_avg" : [hausdorff_avg],
                "hdf_wt" : [hausdorff_wt],
                "hdf_tc" : [hausdorff_tc],
                "hdf_et" : [hausdorff_et],
            }
        }
   

In [204]:
result = evaluate_model(model, 200)

In [209]:
result['metrics']

{'case': [200],
 'dice_avg': [0.89803546667099],
 'dice_wt': [0.9219932556152344],
 'dice_tc': [0.8866446614265442],
 'dice_et': [0.8854685425758362],
 'hdf_avg': [35.40738593765395],
 'hdf_wt': [78.77182237323191],
 'hdf_tc': [13.379088160259652],
 'hdf_et': [14.071247279470288]}

## For Loop automating result accumulation

In [113]:
import pandas as pd
import sys

In [91]:
# Some Random Cases (Avoiding Duplication) -> Later expand to more based on lecturer requirements and feedback
random_cases = np.unique(np.random.randint(0,360,30)).tolist()

In [115]:
# Initialise DF to accumulate results

metric_df = pd.DataFrame()
features_df = pd.DataFrame()

i = 0


for case in random_cases:
    # Do the analysis for 30
    try:

        result = evaluate_model(model, case)

        metric_df = pd.concat([metric_df, pd.DataFrame(result["metrics"])],ignore_index=True)

        ground_truth = get_features(result["input"])

        pred_input = {
            "image": result["input"]["image"].to("cpu"),
            "seg": torch.squeeze(result["pred"]).to("cpu"),
        }

        predicted_features = get_features(pred_input)


        features = pd.DataFrame(
            {
                "case": [case],
                # Whole tumor non textural
                "wt_blobs_original": [ground_truth["blobs"]["blob_wt"]],
                "wt_center_original": [ground_truth["non-textural"]["wt"]["center"]],
                "wt_volume_original": [ground_truth["non-textural"]["wt"]["volume"]],
                "wt_blobs_pred": [predicted_features["blobs"]["blob_wt"]],
                "wt_center_pred": [predicted_features["non-textural"]["wt"]["center"]],
                "wt_volume_pred": [predicted_features["non-textural"]["wt"]["volume"]],
                # Tumor Core non textural
                "tc_blobs_original": [ground_truth["blobs"]["blob_tc"]],
                "tc_center_original": [ground_truth["non-textural"]["tc"]["center"]],
                "tc_volume_original": [ground_truth["non-textural"]["tc"]["volume"]],
                "tc_blobs_pred": [predicted_features["blobs"]["blob_tc"]],
                "tc_center_pred": [predicted_features["non-textural"]["tc"]["center"]],
                "tc_volume_pred": [predicted_features["non-textural"]["tc"]["volume"]],
                # Enhancing Core non textural
                "et_blobs_original": [ground_truth["blobs"]["blob_et"]],
                "et_center_original": [ground_truth["non-textural"]["et"]["center"]],
                "et_volume_original": [ground_truth["non-textural"]["et"]["volume"]],
                "et_blobs_pred": [predicted_features["blobs"]["blob_et"]],
                "et_center_pred": [predicted_features["non-textural"]["et"]["center"]],
                "et_volume_pred": [predicted_features["non-textural"]["et"]["volume"]],
                # Textural Features:
                ## T1
                "tex_energy_t1_original": [ground_truth["textural"]["t1"]["energy"]],
                "tex_homogeneity_t1_original": [
                    ground_truth["textural"]["t1"]["energy"]
                ],
                "tex_contrast_t1_original": [ground_truth["textural"]["t1"]["energy"]],
                "tex_entropy_t1_original": [ground_truth["textural"]["t1"]["energy"]],
                "tex_correlation_t1_original": [
                    ground_truth["textural"]["t1"]["energy"]
                ],
                "tex_energy_t1_pred": [predicted_features["textural"]["t1"]["energy"]],
                "tex_homogeneity_t1_pred": [
                    predicted_features["textural"]["t1"]["energy"]
                ],
                "tex_contrast_t1_pred": [
                    predicted_features["textural"]["t1"]["energy"]
                ],
                "tex_entropy_t1_pred": [predicted_features["textural"]["t1"]["energy"]],
                "tex_correlation_t1_pred": [
                    predicted_features["textural"]["t1"]["energy"]
                ],
                ##T1CE
                "tex_energy_t1ce_original": [
                    ground_truth["textural"]["t1ce"]["energy"]
                ],
                "tex_homogeneity_t1ce_original": [
                    ground_truth["textural"]["t1ce"]["energy"]
                ],
                "tex_contrast_t1ce_original": [
                    ground_truth["textural"]["t1ce"]["energy"]
                ],
                "tex_entropy_t1ce_original": [
                    ground_truth["textural"]["t1ce"]["energy"]
                ],
                "tex_correlation_t1ce_original": [
                    ground_truth["textural"]["t1ce"]["energy"]
                ],
                "tex_energy_t1ce_pred": [
                    predicted_features["textural"]["t1ce"]["energy"]
                ],
                "tex_homogeneity_t1ce_pred": [
                    predicted_features["textural"]["t1ce"]["energy"]
                ],
                "tex_contrast_t1ce_pred": [
                    predicted_features["textural"]["t1ce"]["energy"]
                ],
                "tex_entropy_t1ce_pred": [
                    predicted_features["textural"]["t1ce"]["energy"]
                ],
                "tex_correlation_t1ce_pred": [
                    predicted_features["textural"]["t1ce"]["energy"]
                ],
                ##T2
                "tex_energy_t2_original": [ground_truth["textural"]["t2"]["energy"]],
                "tex_homogeneity_t2_original": [
                    ground_truth["textural"]["t2"]["energy"]
                ],
                "tex_contrast_t2_original": [ground_truth["textural"]["t2"]["energy"]],
                "tex_entropy_t2_original": [ground_truth["textural"]["t2"]["energy"]],
                "tex_correlation_t2_original": [
                    ground_truth["textural"]["t2"]["energy"]
                ],
                "tex_energy_t2_pred": [predicted_features["textural"]["t2"]["energy"]],
                "tex_homogeneity_t2_pred": [
                    predicted_features["textural"]["t2"]["energy"]
                ],
                "tex_contrast_t2_pred": [
                    predicted_features["textural"]["t2"]["energy"]
                ],
                "tex_entropy_t2_pred": [predicted_features["textural"]["t2"]["energy"]],
                "tex_correlation_t2_pred": [
                    predicted_features["textural"]["t2"]["energy"]
                ],
                ##FLAIR
                "tex_energy_flair_original": [
                    ground_truth["textural"]["flair"]["energy"]
                ],
                "tex_homogeneity_flair_original": [
                    ground_truth["textural"]["flair"]["energy"]
                ],
                "tex_contrast_flair_original": [
                    ground_truth["textural"]["flair"]["energy"]
                ],
                "tex_entropy_flair_original": [
                    ground_truth["textural"]["flair"]["energy"]
                ],
                "tex_correlation_flair_original": [
                    ground_truth["textural"]["flair"]["energy"]
                ],
                "tex_energy_flair_pred": [
                    predicted_features["textural"]["flair"]["energy"]
                ],
                "tex_homogeneity_flair_pred": [
                    predicted_features["textural"]["flair"]["energy"]
                ],
                "tex_contrast_flair_pred": [
                    predicted_features["textural"]["flair"]["energy"]
                ],
                "tex_entropy_flair_pred": [
                    predicted_features["textural"]["flair"]["energy"]
                ],
                "tex_correlation_flair_pred": [
                    predicted_features["textural"]["flair"]["energy"]
                ],
            }
        )


        features_df = pd.concat([features_df, features],ignore_index=True)

    except:
        print(f"Error processing case: {case}\n")
        print("Unexpected error:", sys.exc_info()[0])
        raise


c:\Users\ibrah\miniconda3\envs\fyp\lib\site-packages\monai\metrics\hausdorff_distance.py:168: UserWarning: the ground truth of class 2 is all 0, this may result in nan/inf distance.
  warnings.warn(f"the ground truth of class {c} is all 0, this may result in nan/inf distance.")


Error processing case: 289

Unexpected error: <class 'IndexError'>


IndexError: list index out of range

In [44]:
metric_df.sort_values(by='case',inplace=True)

### DICE Scores and Hausdaurff Distance for Each Evaluated Case

In [93]:
metric_df

,case,dice_avg,dice_wt,dice_tc,dice_et,hdf_avg,hdf_wt,hdf_tc,hdf_et
0,3,0.833346,0.732529,0.916864,0.850645,29.954033,76.032888,6.480741,7.348469
1,30,0.806884,0.826244,0.812753,0.781653,40.005474,69.028979,24.738634,26.248809
2,36,0.845233,0.827933,0.885761,0.822005,32.276219,86.272823,5.656854,4.898979
3,42,0.902013,0.939058,0.930642,0.836338,22.597714,51.361464,10.954451,5.477226
4,47,0.879518,0.878071,0.925334,0.835148,51.255711,92.010869,31.080541,30.675723
5,61,0.820444,0.761602,0.908783,0.790947,18.081069,45.188494,4.582576,4.472136
6,62,0.925625,0.933668,0.951146,0.892061,8.544027,11.045361,10.344080,4.242641
7,67,0.926374,0.941353,0.952307,0.885461,24.331659,56.480085,10.770330,5.744563
8,86,0.672317,0.720867,0.642857,0.653226,38.903515,105.280578,4.358899,7.071068
9,90,0.916448,0.894363,0.951745,0.903237,12.688307,27.676705,5.916080,4.472136


In [122]:
metric_df.iloc[:,1:][metric_df.dice_et > 0 ].describe()

,dice_avg,dice_wt,dice_tc,dice_et,hdf_avg,hdf_wt,hdf_tc,hdf_et
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,0.883456,0.888282,0.906479,0.855607,24.283677,49.127096,13.848914,9.875022
std,0.058803,0.075626,0.068617,0.064306,12.358090,29.281668,13.022314,8.331569
min,0.672317,0.720867,0.642857,0.653226,7.344212,9.949874,1.414214,1.732051
25%,0.880296,0.853002,0.893038,0.835743,13.168033,21.853962,6.281752,5.610894
50%,0.901197,0.911046,0.925334,0.876030,22.597714,51.361464,10.344080,7.071068
75%,0.917198,0.945486,0.949435,0.891450,32.197204,72.530934,15.673129,10.073223
max,0.958388,0.969876,0.961973,0.943314,51.255711,105.280578,60.646517,32.649655


### Textural and Non Textural Comparison between ground truth and prediction

In [49]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   case                            4 non-null      int64  
 1   wt_blobs_original               4 non-null      int64  
 2   wt_center_original              4 non-null      object 
 3   wt_volume_original              4 non-null      float64
 4   wt_blobs_pred                   4 non-null      int64  
 5   wt_center_pred                  4 non-null      object 
 6   wt_volume_pred                  4 non-null      float64
 7   tc_blobs_original               4 non-null      int64  
 8   tc_center_original              4 non-null      object 
 9   tc_volume_original              4 non-null      float64
 10  tc_blobs_pred                   4 non-null      int64  
 11  tc_center_pred                  4 non-null      object 
 12  tc_volume_pred                  4 non-nu

#### Difference in Number of Blobs

In [95]:
diff_blobs_wt = features_df.wt_blobs_original - features_df.wt_blobs_pred
diff_blobs_wt.describe()

count    27.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

In [96]:
diff_blobs_tc = features_df.tc_blobs_original - features_df.tc_blobs_pred
diff_blobs_tc.describe()

count    27.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

In [97]:
diff_blobs_et = features_df.et_blobs_original - features_df.et_blobs_pred
diff_blobs_et.describe()

count    27.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

### Comparison of Center point

Only Largest Blob taken as in the off chance that there are 2

In [54]:
def distance_between_midpoints(p1, p2):
            """Given 2 3D points calculate euclidean distance between them.
            Coordinates in (x,y,z)

            Args:
                p1 (tuple): point 1
                p2 (tuple): point 2

            Returns:
                _type_: distance
            """
            return  np.sqrt(p2[0] - p1[0] ** 2 + (p2[1] - p1[1]) ** 2 + (p2[2] - p1[2]) ** 2)

In [98]:
wt_center_offset = features_df.apply(
    lambda row: distance_between_midpoints(row['wt_center_original'], row['wt_center_pred']), axis=1
)
wt_center_offset.describe()

count    27.000000
mean     32.962659
std      30.723328
min       0.888649
25%       3.900877
50%      30.689826
75%      47.801052
max      98.624723
dtype: float64

In [99]:
tc_center_offset = features_df.apply(
    lambda row: distance_between_midpoints(row['tc_center_original'], row['tc_center_pred']), axis=1
)
tc_center_offset.describe()

count     27.000000
mean       9.011370
std       22.928087
min        0.345896
25%        1.332717
50%        2.078482
75%        2.877048
max      111.427613
dtype: float64

In [101]:
et_center_offset = features_df.apply(
    lambda row: distance_between_midpoints(row['et_center_original'], row['et_center_pred']), axis=1
)
et_center_offset.describe()

count     27.000000
mean       7.315402
std       21.358199
min        0.398660
25%        1.440253
50%        2.005945
75%        3.157804
max      112.412641
dtype: float64

### Comparison of Difference in Tumor Volume

In [102]:
diff_volume_wt = np.abs(features_df.wt_volume_original - features_df.wt_volume_pred)
diff_volume_wt.describe()       

count    27.000000
mean      8.699222
std      11.799127
min       0.193000
25%       1.548500
50%       3.886000
75%      11.686500
max      53.542000
dtype: float64

In [103]:
diff_volume_tc = np.abs(features_df.tc_volume_original - features_df.wt_volume_pred)
diff_volume_tc.describe()

count     27.000000
mean      57.325815
std       39.362460
min       11.352000
25%       26.218000
50%       42.122000
75%       82.659000
max      136.630000
dtype: float64

In [104]:
diff_volume_et = np.abs(features_df.et_volume_original - features_df.wt_volume_pred)
diff_volume_et.describe()   

count     27.000000
mean      71.631074
std       45.735554
min       13.799000
25%       30.142500
50%       63.535000
75%      104.856500
max      153.745000
dtype: float64

### Comparison of % difference of textural features.

$$\frac{original-pred}{original}\cdot100$$

Direction Vector Used was $(1,1,1)$

In [105]:
diff_tex_t1 = pd.DataFrame()
diff_tex_t1.loc[:,"Energy"] = np.abs(features_df.tex_energy_t1_original - features_df.tex_energy_t1_pred) / features_df.tex_energy_t1_original * 100


diff_tex_t1.loc[:,"Homogeneity"] = np.abs(features_df.tex_homogeneity_t1_original - features_df.tex_homogeneity_t1_pred) / features_df.tex_homogeneity_t1_original * 100

diff_tex_t1.loc[:,"Contrast"] = np.abs(features_df.tex_contrast_t1_original - features_df.tex_contrast_t1_pred) / features_df.tex_contrast_t1_original * 100

diff_tex_t1.loc[:,"Entropy"] = np.abs(features_df.tex_entropy_t1_original - features_df.tex_entropy_t1_pred) / features_df.tex_entropy_t1_original * 100

diff_tex_t1.loc[:,"Correlation"] = np.abs(features_df.tex_correlation_t1_original - features_df.tex_correlation_t1_pred) / features_df.tex_correlation_t1_original * 100


In [106]:
diff_tex_t1.describe()

,Energy,Homogeneity,Contrast,Entropy,Correlation
count,27.000000,27.000000,27.000000,27.000000,27.000000
mean,287.396657,287.396657,287.396657,287.396657,287.396657
std,726.020139,726.020139,726.020139,726.020139,726.020139
min,1.935273,1.935273,1.935273,1.935273,1.935273
25%,13.390527,13.390527,13.390527,13.390527,13.390527
50%,25.406454,25.406454,25.406454,25.406454,25.406454
75%,103.683027,103.683027,103.683027,103.683027,103.683027
max,2785.197446,2785.197446,2785.197446,2785.197446,2785.197446


In [107]:
diff_tex_t1ce = pd.DataFrame()
diff_tex_t1ce.loc[:,"Energy"] = np.abs(features_df.tex_energy_t1ce_original - features_df.tex_energy_t1ce_pred) / features_df.tex_energy_t1ce_original * 100


diff_tex_t1ce.loc[:,"Homogeneity"] = np.abs(features_df.tex_homogeneity_t1ce_original - features_df.tex_homogeneity_t1ce_pred) / features_df.tex_homogeneity_t1ce_original * 100

diff_tex_t1ce.loc[:,"Contrast"] = np.abs(features_df.tex_contrast_t1ce_original - features_df.tex_contrast_t1ce_pred) / features_df.tex_contrast_t1ce_original * 100

diff_tex_t1ce.loc[:,"Entropy"] = np.abs(features_df.tex_entropy_t1ce_original - features_df.tex_entropy_t1ce_pred) / features_df.tex_entropy_t1ce_original * 100

diff_tex_t1ce.loc[:,"Correlation"] = np.abs(features_df.tex_correlation_t1ce_original - features_df.tex_correlation_t1ce_pred) / features_df.tex_correlation_t1ce_original * 100

In [108]:
diff_tex_t1ce.describe()

,Energy,Homogeneity,Contrast,Entropy,Correlation
count,27.000000,27.000000,27.000000,27.000000,27.000000
mean,289.128447,289.128447,289.128447,289.128447,289.128447
std,731.341176,731.341176,731.341176,731.341176,731.341176
min,1.936703,1.936703,1.936703,1.936703,1.936703
25%,14.010748,14.010748,14.010748,14.010748,14.010748
50%,24.922507,24.922507,24.922507,24.922507,24.922507
75%,104.049017,104.049017,104.049017,104.049017,104.049017
max,2818.279044,2818.279044,2818.279044,2818.279044,2818.279044


In [109]:
diff_tex_t2 = pd.DataFrame()
diff_tex_t2.loc[:,"Energy"] = np.abs(features_df.tex_energy_t2_original - features_df.tex_energy_t2_pred) / features_df.tex_energy_t2_original * 100


diff_tex_t2.loc[:,"Homogeneity"] = np.abs(features_df.tex_homogeneity_t2_original - features_df.tex_homogeneity_t2_pred) / features_df.tex_homogeneity_t2_original * 100

diff_tex_t2.loc[:,"Contrast"] = np.abs(features_df.tex_contrast_t2_original - features_df.tex_contrast_t2_pred) / features_df.tex_contrast_t2_original * 100

diff_tex_t2.loc[:,"Entropy"] = np.abs(features_df.tex_entropy_t2_original - features_df.tex_entropy_t2_pred) / features_df.tex_entropy_t2_original * 100

diff_tex_t2.loc[:,"Correlation"] = np.abs(features_df.tex_correlation_t2_original - features_df.tex_correlation_t2_pred) / features_df.tex_correlation_t2_original * 100

In [110]:
diff_tex_t2.describe()

,Energy,Homogeneity,Contrast,Entropy,Correlation
count,27.000000,27.000000,27.000000,27.000000,27.000000
mean,283.970914,283.970914,283.970914,283.970914,283.970914
std,718.948321,718.948321,718.948321,718.948321,718.948321
min,1.937831,1.937831,1.937831,1.937831,1.937831
25%,14.228744,14.228744,14.228744,14.228744,14.228744
50%,25.428907,25.428907,25.428907,25.428907,25.428907
75%,103.732404,103.732404,103.732404,103.732404,103.732404
max,2712.248818,2712.248818,2712.248818,2712.248818,2712.248818


In [111]:
diff_tex_flair = pd.DataFrame()
diff_tex_flair.loc[:,"Energy"] = np.abs(features_df.tex_energy_flair_original - features_df.tex_energy_flair_pred) / features_df.tex_energy_flair_original * 100


diff_tex_flair.loc[:,"Homogeneity"] = np.abs(features_df.tex_homogeneity_flair_original - features_df.tex_homogeneity_flair_pred) / features_df.tex_homogeneity_flair_original * 100

diff_tex_flair.loc[:,"Contrast"] = np.abs(features_df.tex_contrast_flair_original - features_df.tex_contrast_flair_pred) / features_df.tex_contrast_flair_original * 100

diff_tex_flair.loc[:,"Entropy"] = np.abs(features_df.tex_entropy_flair_original - features_df.tex_entropy_flair_pred) / features_df.tex_entropy_flair_original * 100

diff_tex_flair.loc[:,"Correlation"] = np.abs(features_df.tex_correlation_flair_original - features_df.tex_correlation_flair_pred) / features_df.tex_correlation_flair_original * 100

In [112]:
diff_tex_flair.describe()

,Energy,Homogeneity,Contrast,Entropy,Correlation
count,27.000000,27.000000,27.000000,27.000000,27.000000
mean,289.124697,289.124697,289.124697,289.124697,289.124697
std,727.874363,727.874363,727.874363,727.874363,727.874363
min,1.942599,1.942599,1.942599,1.942599,1.942599
25%,17.441094,17.441094,17.441094,17.441094,17.441094
50%,25.299943,25.299943,25.299943,25.299943,25.299943
75%,110.075532,110.075532,110.075532,110.075532,110.075532
max,2814.091438,2814.091438,2814.091438,2814.091438,2814.091438
